In [ ]:
import pandas as pd
import openpyxl
import sqlite3

sh_users = pd.read_excel('users.xlsx')
sh_users.sort_values(by="name", inplace=True)

conn = sqlite3.connect('users.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM users ORDER BY name ASC')
db_users = cursor.fetchall()

conn.commit()
conn.close()

column_names = [description[0] for description in cursor.description]
db_users = pd.DataFrame(db_users, columns=column_names)

sh_users.info(), db_users.info()

In [ ]:
exact_match = sh_users.equals(db_users)

if not exact_match:
    # create 2 dataframes to compare
    old_users = pd.merge(sh_users, db_users, how="outer", indicator=True).query('_merge == "right_only"').drop(columns='_merge')
    new_users = pd.merge(sh_users, db_users, how="outer", indicator=True).query('_merge == "left_only"').drop(columns='_merge')

db_users

In [ ]:
old_users

In [ ]:
new_users

In [ ]:
old_users_tuples = [tuple(old_user) for old_user in old_users.values]
new_users_tuples = [tuple(new_user) for new_user in new_users.values]

conn = sqlite3.connect('users.db')
cursor = conn.cursor()

for name, number, email, reporting_officer, hod in old_users_tuples:
    cursor.execute('DELETE FROM users WHERE name = ?', (name,))
    
for name, number, email, reporting_officer, hod in new_users_tuples:
    cursor.execute('INSERT INTO users (name, number, email, reporting_officer, hod) VALUES (?, ?, ?, ?, ?)', (name, number, email, reporting_officer, hod))

conn.commit()
conn.close()

In [ ]:
import datetime
import os
import requests
from dotenv import load_dotenv

load_dotenv("../.env")

In [1]:
import datetime
import os
import requests
from dotenv import load_dotenv

load_dotenv("../.env")

class AzureSyncError(Exception):

    def __init__(self, message):
        self.message = message
        super().__init__(self.message)

def cur_datetime(dt_type):
    return datetime.datetime.now().strftime(dt_type)

def get_filename():
    cur_date = datetime.datetime.now()
    format_date = cur_date.strftime("%Y")

    return format_date

template_path = os.path.join(os.path.dirname(os.getcwd()), 'excel_files', 'mc_template.xlsx')

drive_url = f"https://graph.microsoft.com/v1.0/drives/{os.environ.get('DRIVE_ID')}"

folder_url = drive_url + f"/items/{os.environ.get('FOLDER_ID')}"

book_path = folder_url + f"/children?$filter=name eq '{get_filename()}.xlsx'"
create_book_url = folder_url + f":/{get_filename()}.xlsx:/content"

config = {
    'client_id': os.environ.get('CLIENT_ID'),
    'client_secret': os.environ.get('CLIENT_SECRET'),
    'authority': os.environ.get('AUTHORITY'),
    'scope': [os.environ.get('SCOPE')],
    'site_id': os.environ.get('SITE_ID'),
}

headers = {
        'Authorization': f'Bearer {os.environ.get("ACCESS_TOKEN")}',
        'Content-Type': 'application/json'
    }



def get_table_url(worksheet_url):

    tables_url = f"{worksheet_url}/tables"
    print(tables_url)


    # get the tables
    response = requests.get(url=tables_url, headers=headers)
    if not 200 <= response.status_code < 300:
        raise AzureSyncError(f"Could not get the tables. {response.json()}")

    # if table not found, add_table() creates one
    tables = response.json()['value']

    table_ids = {table_obj["name"]: table_obj['id'] for table_obj in response.json()['value']}
    
    if cur_datetime("%B") not in table_ids.keys():
        table_id = add_table(worksheet_url, cur_datetime("%B"))
    # if table found, get the id
    else:
        table_id = table_ids[f"{cur_datetime('%B')}"]
        
    table_url = f"{tables_url}/{table_id}"

    return table_url

def add_table(worksheet_url, name):

    # ADD TABLE HEADERS
    table_headers_url = f"{worksheet_url}/range(address='A1:B1')"

    header_values = {
        "values": [["Name", "Date"]]
    }

    response = requests.patch(table_headers_url, headers=headers, json=header_values)
    if not 200 <= response.status_code < 300:
        raise AzureSyncError(f"Table headers could not be initialised. {response.json()}")

    # ADD TABLE
    add_table_url = f"{worksheet_url}/tables/add"

    body = {
        "address": "A1:B1",
        "hasHeaders": True,
    }

    response = requests.post(add_table_url, headers=headers, json=body)
    if not 200 <= response.status_code < 300:
        raise AzureSyncError(f"Table itself could not be initialised. {response.json()}")
    table_id = response.json()['id']

    # CHANGE TABLE NAME
    change_tablename_url = f"{worksheet_url}/tables/{table_id}"

    table_options = {
        "name": name
    }

    response = requests.patch(change_tablename_url, headers=headers, json=table_options)
    if not 200 <= response.status_code < 300:
        raise AzureSyncError(f"Table name could not be changed. There might be tables with duplicate names {response.json()}")

    # return table
    table_id = response.json()['id'] 

    return table_id

def add_worksheet(worksheets_url, name):


    body = {
        "name": f"{name}"
    }

    # add the worksheet
    response = requests.post(url=f"{worksheets_url}/add", headers=headers, json=body)

    if 200 <= response.status_code < 300:
        print("sheet added successfully!")
        worksheet_id = response.json()['id']
    else:
        raise AzureSyncError(f"Sheet failed to add. Status code: {response.status_code}")
        print(response.json())

    return worksheet_id

def get_sheet_url(worksheets_url):

    # get the worksheet names
    response = requests.get(url=worksheets_url, headers=headers)

    # if ws not found, add_worksheet() creates one

    # get the worksheet names
    response = requests.get(url=worksheets_url, headers=headers)

    ws_ids = {sheet_obj["name"]: sheet_obj['id'] for sheet_obj in response.json()['value']}
    
    if cur_datetime('%B') not in ws_ids.keys():
        ws_id = add_worksheet(worksheets_url, cur_datetime("%B"))
    # if ws found, get the id
    else:
        ws_id = ws_ids[f"{cur_datetime('%B')}"]
        
    worksheet_url = f"{worksheets_url}/{ws_id}"

    return worksheet_url

def write_to_excel(write = True): #TODO add params

    new_book = False


    try:
        # get the workbook for this month
        cur_book = requests.get(url=book_path, headers=headers) 

        file_name = cur_book.json()['value']
        if not file_name:
            new_book = True
            book_id = create_book()
        else:
            book_id = file_name[0]['id']

        print(book_id)

        worksheets_url = drive_url + f"/items/{book_id}/workbook/worksheets"

        # checks if sheet for this month exists, otherwise create it
        worksheet_url = get_sheet_url(worksheets_url)
        print(f"Worksheet URL FINAL = {worksheet_url}")

        table_url = get_table_url(worksheet_url)
        print(f"Table URL FINAL = {table_url}")

        if write is not False:

            # delete Sheet1
            if new_book == True:
                deleteSheet1(worksheets_url)

            # write to file
            write_to_table_url = f"{table_url}/rows"

            body = {
                "values": [
                    ["Rachmiel", "'11/11/2023"],
                    ["Shawn", "'11/11/2023"],
                    ["Rachmiel", "'12/11/2023"],
                    ["Tymothy", "'12/11/2023"]
                ]
            }

            response = requests.post(write_to_table_url, headers= headers, json = body)

            if 200 <= response.status_code < 300:
                print("Data uploaded successfully!")
            else:
                raise AzureSyncError(f"Failed to upload data. Status code: {response.status_code}, {response.json()}")
        
        return table_url
        

    except AzureSyncError as e:
        print(e.message)
        return

def create_book():

    with open(template_path, 'rb') as file_data:

        # uploads the file
        response = requests.put(create_book_url, headers=headers, data=file_data) 

    if 200 <= response.status_code < 300:
        print("File added successfully!")
        book_id = response.json()['id']
        return book_id
    else:
        raise AzureSyncError(f"Failed to upload file. Status code: {response.status_code}, {response.json()}")
    

def deleteSheet1(worksheets_url):

    del_sheet1_url = worksheets_url + "/Sheet1"

    # delete the sheet
    response = requests.delete(del_sheet1_url, headers=headers)
    if 200 <= response.status_code < 300:
        print("Sheet successfully deleted")
        return
    else:
        raise AzureSyncError(f"Something went wrong when creating the worksheet. Status code: {response.status_code}, {response.json()}")

In [2]:
write_to_excel(True)

File added successfully!
01JE235THHXA57JUY5OZDKLYLFN5FJ2CEZ
sheet added successfully!
Worksheet URL FINAL = https://graph.microsoft.com/v1.0/drives/b!miY83vtC4E6hZVysK2MuIIXS4kkbArdAu4a1mtl-tW_FUmaJ5dR7Sq1_9ZAZ_eG9/items/01JE235THHXA57JUY5OZDKLYLFN5FJ2CEZ/workbook/worksheets/{4ADDF0D8-F0F7-4F6F-8303-3737513886C7}
https://graph.microsoft.com/v1.0/drives/b!miY83vtC4E6hZVysK2MuIIXS4kkbArdAu4a1mtl-tW_FUmaJ5dR7Sq1_9ZAZ_eG9/items/01JE235THHXA57JUY5OZDKLYLFN5FJ2CEZ/workbook/worksheets/{4ADDF0D8-F0F7-4F6F-8303-3737513886C7}/tables
Could not get the tables. {'error': {'code': 'ItemNotFound', 'message': "The requested resource doesn't exist.", 'innerError': {'code': 'itemNotFound', 'message': "The requested resource doesn't exist.", 'date': '2023-11-02T09:29:55', 'request-id': 'c2539516-eee4-4310-8f85-7adb14486900', 'client-request-id': 'c2539516-eee4-4310-8f85-7adb14486900'}}}


In [ ]:
month = datetime.datetime.now().strftime("%B")
month

In [ ]:
import pandas as pd

table_url = write_to_excel(False)

def send_message_to_principal():
        response = requests.get(url=f"{table_url}/rows?", headers=headers)
        mc_arrs = [tuple(info) for object_info in response.json()['value'] for info in object_info['values']]
        mc_table = pd.DataFrame(data = mc_arrs, columns=["name", "date"])
        date_today = datetime.datetime.strftime(datetime.datetime.now(), "%d/%m/%Y")
        mc_today = mc_table.loc[mc_table['date'] == date_today]
        mc_tuples = [tuple(new_user) for new_user in mc_today.values]
        print(mc_tuples)
        message = f"Staff on MC on {date_today}\n:"
        for user, date in mc_tuples:
                message += f"{user}\n"
        return message

mc_tuples = send_message_to_principal()
mc_tuples